In [1]:
# open a .txt containing the extracted comments from MS Word
trackchange_doc = open("data/reconciled_annotations.txt",'r').read().strip().split("\n")

position_of_comments = []
comments_abstract_match = {}

# we loop over the file
for e in range(len(trackchange_doc)):
    element = trackchange_doc[e].strip()
    # if the line indicates a commeent
    if "Commented" in element:        
        # we save the line number and the page it refers to
        number = element.split(":")[0].replace("Page ","")
        if number.isnumeric():
            position_of_comments.append(e)
            comments_abstract_match[e] = number
            
# this is the number of comments we found
print (len(position_of_comments))

852


In [2]:
# we save the annotations in a spreadsheet
out = open("outputs/extracted_reconciled_annotations.tsv","w")
out.write("comment_id\tabstract_id\tannotator\tscore\tcomment\n")
# this dictionary matches the name of the annotators as MS Word users with their text mention
authors = {"dey,":"Dey,  Gautam", "jessica":"Jessica  Polka","mate":"Mate  Palfy","jonathon":"jonathon coates"}


# we loop over the comments
for c in range(len(position_of_comments)):
    # we retrieve the position of the comment and the related abstract id (its page)
    comment_pos = position_of_comments[c]
    abstract_id = comments_abstract_match[comment_pos]
    
    # here we know where to stop, at the beginning of the next change (or at the end of the doc)
    if c+1 in position_of_comments:
        nextchange = position_of_comments[c+1]
    else:
        nextchange = len(trackchange_doc)+1
    
    #Page 1: Commented [2] jonathon coates 7/24/2020 1:07:00 PM  Context_effect-_1-  
    
    # this is the "comment id" as in MS Word
    id_ = "["+trackchange_doc[comment_pos].split("[")[1].split("]")[0]+"]"
    
    # we try to find the author as, depending on the format is in one of the next lines
    possible_author = trackchange_doc[comment_pos].split("]")[1].strip().split(" ")[0].strip().lower()
    author = authors[possible_author]
    annotation = trackchange_doc[comment_pos].strip().split(" ")[-1]
    comment = ""
    if "_" not in annotation:
        check = False
        n_tokens = len(trackchange_doc[comment_pos].split(" "))
        for x in reversed(range(0,n_tokens)):
            if check is False:
                token = trackchange_doc[comment_pos].split(" ")[x]
                if "_" in token:
                    comment = trackchange_doc[comment_pos].strip().split(token)[1]
                    annotation = token
                    check = True
            else:
                break
    if "_" not in annotation:
        following_line = trackchange_doc[comment_pos+1]
        if "Page" not in following_line and "_" in following_line:
            annotation = [x for x in following_line.split(" ") if "_" in x][0]
            comment = following_line.replace(annotation,"")
    if "_" in annotation:    
        out.write(id_+"\t"+abstract_id+"\t"+author+"\t"+annotation.strip().lower()+"\t"+comment+"\n")

out.close()